Author: Mit Gor

#### This file contains several tests of methods of applyng textures to an image, in order to use them as part of the final accretion disk image for the project.

In [ ]:
pip install TensorFlow tensorflow_hub 

In [ ]:
pip install --upgrade jax jaxlib

In [ ]:
from PIL import Image
from PIL import Image, ImageEnhance, ImageChops
import cv2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.image as mpimg_saver


import functools
import os

import tensorflow as tf
import tensorflow_hub as hub



In [ ]:
#Here is an example of one of the many different coding techniques that were used to experiment with
#different textures, effects and texts

#loading the original image and the texture image (in this example it is smoke)
original_base = Image.open("blank_disc (3) (1).jpg") 
texture_smoke = Image.open("texture5.png")

#here the modes are being converted to match so the image processes can take place without errors
if original_base.mode != texture_smoke.mode:
    texture_smoke = texture_smoke.convert(original_base.mode)

#the texture image is resized to match the original so the effects are enlarged over the whole image
texture_smoke = texture_smoke.resize(original_base.size)

#stock empty image used to work one same sizes and modes as base
new_image = Image.new(original_base.mode, original_base.size, (0, 0, 0))

#blending the two images together 
blend_image = ImageChops.overlay(original_base, texture_smoke)

#now the blended image is put over the empty one
new_image.paste(blend_image, (0, 0))

#these are enhancer factors that were played around and experimented for the best looking final image
enhancer = ImageEnhance.Brightness(new_image)

#these factors are all dependent on the texture being used
new_image = enhancer.enhance(1.2)

#for the smoke one the saturation didn't have to be that high but for the saturn ring textures they did etc
enhancer = ImageEnhance.Color(new_image)
new_image = enhancer.enhance(1.2) #saturation 

#saving the resulting frame
new_image.save("new_image.png")


In [ ]:
#another example of the various methods on python used to experiment with textures and effects:

#here a grayscale wavelength image is being used as the base so the codes specific to these requirements
original_img = Image.open('wavelength.jpg').convert('L')
#converting the modes to match and corresponding arrays
original_img_array = np.array(original_img)

#the texture image is loaded and arrays created for image analysis
new_texture = Image.open('texture.jpg').convert('RGB')
new_texture_array = np.array(new_texture)

#the sizes are resized and matched to each other, making sure the heights and widths of the texture is the same
height, width = original_img_array.shape
#otherwise the texture would only be created on one section of its own size
new_texture_resized = new_texture.resize((width, height), resample=Image.Resampling.BILINEAR)
#bilinear interpolation used for the resampling experiments here, it interpolates the pixels based on neighbour means
#found it to produce better and smoother looking results

new_texture_resized_array = np.array(new_texture_resized)


#here a mask is made and the numbers are continualously changed based on what sections of the image
#the texture should be applied to
mask_location = np.zeros_like(new_texture_resized)
#texture effect in this example is applied everywhere on the base image 
mask_location[new_texture_resized_array > 50] = 1

#the texture is now blended to the base image
alpha = 0.5 
#alpha allows the opacity to change which makes the final image more "blended" and smoother
texture_blend = original_img_array[..., np.newaxis] * (1 - alpha) + new_texture_resized_array * alpha
#blending occurs based on the opacity and a final blended array created from the two 
texture_blend = np.clip(texture_blend, 0, 255) 
#ensuring the pixel values are within a range(experimented on and changed here and there)

#converted to grayscale image by takin mean of the RGB channels 
output_array = np.mean(texture_blend, axis=2)
#final image objects created to be saved 
final_blended_img = Image.fromarray(np.uint8(output_array))

#final image with the texture blended on the original image with improvements made saved
final_blended_img.save('output.jpg')


In [ ]:
#here is a code utilising the neural style transfer method
#it is not from scratch and is using tutorial code with a module linked below(in hub_handle)


#loading the hub being used for the image stylisation
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

#function for loading the image taking its path and size (this is same as the one used in the tutorial code)
def load_image(path, image_size):
    img_saver = tf.io.read_file(path)
    #image path read
    img_saver = tf.image.decode_jpeg(img_saver, channels=3)
    #read as a jpg file and type converted to float
    img_saver = tf.image.convert_image_dtype(img_saver, tf.float32)
    #resized to the image size
    img_saver = tf.image.resize(img_saver, image_size)
    return img_saver[tf.newaxis, :]

#function for saving the image as a jpg
def save_image(image, file_path):
    #here the image is scaled to a specific pixel range (ensure the correct stylised image is saved)
    image = (image[0] * 255).numpy().astype(np.uint8)
    #saved as a jpg 
    image = Image.fromarray(image)
    image.save(file_path, format='JPEG')

#the original base image is loaded 
base_image = 'blank_disc (3) (1).jpg'  
#due to GPU limitations the base image can't be as high as our original image thus the resolution greatly reduced
#this is then fixed post code and stylisation through enhancing techniques and photoshop
#stylisation image also loaded
stylisation_texture = 'style.jpg' 
#size used in hub code but experimented and changed based on images being used
image_size= 500  


base_image_size = (image_size, image_size)
#texture size recommended by hub to stay at 256 but changed depending on whats being worked with
texture_size = (256, 256)  

#the base image is loaded using the function
blended_image2 = load_image(base_image, base_image_size)
#same for stylised image
stylised_image = load_image(stylisation_texture, texture_size)


#average pooling performed to smooth any small variations
#however this reduces the resolution thus effects and photoshop needed to enhance final image 
stylised_image = tf.nn.avg_pool(stylised_image, ksize=[3,3], strides=[1,1], padding='SAME')


#final image tensor made by passing the inputs in the module here the acutal stylisation takes place
final_image = hub_module(tf.constant(blended_image2), tf.constant(stylised_image))

#in the module the gram matrices and equations is calculated for the style and then the stylisation is produced and optimised
#it then returns the stylised image that will be saved

stylized_image = final_image[0]
#extracted with [0] due to tensor containing optimisation loss as well


#here are some post processing techniques that were applied to the final image for experimenting
stylized_image = tf.image.adjust_brightness(stylized_image, delta=0.3)
#mainly used photoshop to change these instead
#since they produced better results and features such as sharpness were not available
stylized_image = tf.image.adjust_contrast(stylized_image, contrast_factor=1.5)
stylized_image = tf.image.adjust_saturation(stylized_image, saturation_factor=1.5)


#image converted to array
img_saver = np.array(stylized_image[0])
#pixel vals clipped to 0-255
img_saver = tf.clip_by_value(img_saver, 0, 255)
#uint data conversion
img_saver = np.uint8(img_saver.numpy())

#final styalised image saved:
save_image(stylized_image, 'stylized_image.jpg')




